In [1]:
import warnings
import datetime
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from os import listdir
from os.path import isfile, join
import os
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.test.utils import common_corpus
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [2]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thiya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
KUNARAC = "Kunarac"
BRIMA = "Brima"
BEMBA = "Bemba"
VICTIMS = "victims"
JUDGEMENTS = "judgements"
WITNESSES = "Witnesses"
DEF_ITERATIONS = 1000
DEF_TOPICS = 10

In [4]:
#os.environ['MALLET_HOME'] = "C:\\Users\\thiya\\OneDrive\\NUS\\Year 3\\Sem 2\\RA\\Martin Buber\\Mallet"

In [5]:
PATH_TO_MALLET = "C:\\Users\\thiya\\OneDrive\\NUS\\Year 3\\Sem 2\\RA\\Martin Buber\\Mallet\\bin\\mallet"
PATH_TO_TXT_MODELS = "C:\\Users\\thiya\\OneDrive\\NUS\\Year 3\\Sem 2\\RA\\Matin Buber\\Results"
DATA_PATH = "C:\\Users\\thiya\\OneDrive\\NUS\\Year 3\\Sem 2\\RA\\Martin Buber\\Data"
PROCESSED_FILES_FOLDER = "C:\\Users\\thiya\\OneDrive\\NUS\\Year 3\\Sem 2\\RA\\Martin Buber\\Processed Files"
STOP_WORDS_PATH = "C:\\Stop Words"

In [6]:
SPECIAL_WORDS = ["was", "us", "as", "less", "has", "media", "Serbs", "muslims", "raped", "man", "commander", "worked",
                 "men", "does", "gave", "happened", "commander", "heard", "judes", "fo"]

stop_words = ["haynes"]

In [7]:
def get_files(dir_path):
    file_names = [name for name in listdir(dir_path) if isfile(join(dir_path, name))]
    all_files = []
    for file_name in file_names:
        print(file_name)
        n_file = open(dir_path + "\\" + file_name, "r", errors='ignore').readlines()
        all_files.append(n_file)
    return all_files

In [8]:
def discard_enumeration(file_list):
    new_files = []
    for file in file_list:
        if file is not None:
            lines = []
            i = 0
            for i in range(len(file)):
                line = file[i].lower()
                # delete lines announcing blank pages
                if line == "Blank page inserted to ensure pagination corresponds between the French and":
                    line = file[i+2].lower()
                if "open session" not in line:
                    if not ("ENG" in line and "CT" in line):
                        line = line.split(" ")
                        if line[0] != '\n':
                            if line[0].isdigit():
                                line = line[1:]
                    lines.append(" ".join(line))
            new_files.append(lines)
    return new_files

In [9]:
def tokenize_files(file_list):
    tokenizer = RegexpTokenizer(r'\w+')
    tokenized = []
    for file in file_list:
        t_file = ""
        for line in file:
            t_file += line
        t_file = tokenizer.tokenize(t_file)
        t_file = [word.lower() for word in t_file]
        tokenized.append(t_file)
    return tokenized
   


In [10]:
def basic_NER(file_list):
    names = {"radomir": "kovac", "klanfa" : "kovac",
             "zaga" : "kunarac", "dragoljub":"kunarac", "zoran" : "vukovic",
              "gullit": "alex-tamba-brima",
             "saj": "musa"}
    double_names = {"rojoj": "pass", "sierra": "leone", "buk": "bijela",
                    "cerova": "ravan", "lepa": "brena", "kp": "dom",
                    "junior": "lion"}
    triple_names = {"santigie": ["borbor", "kanu"]}
    window = 3
    new_files = []
    i = 0
    for file in file_list:
        new_file = []
        while i < len(file):
            word = file[i].lower()
            word_win = file[i - window:i + window]
            word_win = [w.lower() for w in word_win]
            if word == "leone":
                    new_file.append("sierra-leone")
            elif word in names:
                new_file.append(names[word])
            elif word in double_names:
                if double_names[word] in file[i-window:i+window] or "leonean" in file[i-window:i+window]:
                    new_file.append(word + "-" + double_names[word])
                    i += 1
            elif word in triple_names:
                new_file.append(word + "-" + triple_names[word][0] + "-" + triple_names[word][1])
                if triple_names[word][0] in word_win:
                    if triple_names[word][1] in word_win:
                        i += 2
                    else:
                        i += 1
            elif word == "alex":
                if 'brima' in word_win:
                    new_file.append("alex-tamba-brima")
                    i += 2
            elif word == "tamba":
                new_file.append("alex-tamba-brima")
                i+=1
            elif word == "ibrahim":
                if 'bazzy' in word_win or 'kamara' in word_win:
                    new_file.append("brima-kamara")
                    i += 2
            elif word == "sam" or word == "bockarie":
                if "bockarie" in word_win:
                    new_file.append("mosquito")
                    i += 1
            elif word == 'bazzy':
                new_file.append("brima-kamara")
                i += 1
            elif word == "brima":
                if 'et' and 'al' in word_win:
                    i += 2
            else:
                new_file.append(word)
            i+=1
        new_files.append(new_file)
        i = 0
    return new_files

In [11]:
def lemmatize_files(file_list):
    lemmatizer = WordNetLemmatizer()
    lemmatized_files = []
    for file in file_list:
        new_file = []
        for word in file:
            if word.lower() in SPECIAL_WORDS:
                word = special_lemmatizer(word)
            else:
                word = lemmatizer.lemmatize(word)
            new_file.append(word)
        lemmatized_files.append(new_file)
    return lemmatized_files


In [12]:
def special_lemmatizer(word):
    word = word.lower()
    if word.lower() == "raped":
        return "rape"
    if word.lower() == "men":
        return "man"
    if word == "was":
        return "is"
    if word == "commander":
        return "command"
    if word == "worked":
        return "work"
    if word == "muslims":
        return "muslim"
    if word == "Serbs":
        return "Serb"
    if word == "men":
        return "man"
    if word == "gave":
        return "give"
    if word == "does" or word == "doent":
        return "do"
    if word == "happened":
        return "happen"
    if word == "vukovi":
        return "vukovic"
    if word == "herad":
        return "hear"
    if word == "fo":
        return "foca"
    if word =="dragoljub":
        return ""
    else:
        return word

In [13]:
def clean(files_list, stop_words):
    clean_files = []
    for file in files_list:
        c_file = []
        for word in file:
            if word.lower() not in stop_words and "ÿ" not in word and not word.isdigit():
                c_file.append(word.lower())
        clean_files.append(c_file)
    return clean_files

In [14]:
def count_freq(files_list):
    frequencies = {}
    for file in files_list:
        for word in file:
            if word not in frequencies:
                frequencies[word] = 1
            else:
                frequencies[word] += 1
    x = sorted((value, key) for (key, value) in frequencies.items())
    x.reverse()

In [15]:
def check_words(file_list):
    all = {}
    for file in file_list:
        for line in file:
            line = line.split(" ")
            for word in line:
                word = word.strip()
                word = re.sub('[!#?.,-\\\]', '', word)
                word = word.title()
                if str(word) not in all:
                    all[str(word)] = 1
                else:
                    all[str(word)] += 1
    all = sorted((value, key) for (key, value) in all.items())
    all.reverse()
    print(all)
    print(len(all))
    return all

In [16]:
def create_data(path):
    file_list = get_files(path)
    file_list = discard_enumeration(file_list)
    file_list = tokenize_files(file_list)
    file_list = basic_NER(file_list)
    file_list = lemmatize_files(file_list)
    file_list = clean(file_list, stop_words)
    return file_list


In [17]:
def create_lda_model(file_path, case="default", num_topics=10, iterations=5000, save=True):
    dictionary = corpora.Dictionary(file_path)
    corpus = [dictionary.doc2bow(file) for file in file_path]
    model = LdaMallet(PATH_TO_MALLET, corpus=corpus, num_topics=num_topics,
                      id2word=dictionary, iterations=iterations)
    now = datetime.datetime.now()
    if save:
        serial_name = name + "_" + str(num_topics) + "_topics_" + str(now.hour) + "h" + str(now.minute) + "m"
        print("saving as " + serial_name)
        print(" ")
        model.save(serial_name)
        save_topics_to_text(model, serial_name)
    return model

In [18]:
def save_topics_to_text(lda_model, name):
    topics = lda_model.print_topics(num_words=20)
    file = open(PATH_TO_TXT_MODELS + case + "\\" + name + ".txt", 'w+')
    for topic in topics:
        file.write("Topic " + str(topic[0]) + ": \n\n")
        words = topic[1].split("+")
        for i in range(len(words)):
            word = words[i].split("*")
            word = word[1].strip()
            word = word.replace("\"", "")
            file.write(word)
            if i < len(words)-1:
                file.write(", ")
            else:
                file.write(".")
            if i == 9:
                file.write("\n")
        file.write("\n\n")

In [19]:
def show_topics_graph(dictionaries):
    i = 1
    for dict in dictionaries:
        print(dict)
        dict1 = [(v,k) for k,v in dict.items()]
        dict1.sort(reverse=True)
        x = [i for i in range(len(dict))]
        y = [dict[str(i)] for i in range(len(dict))]
        plot = plt.bar(x, y)
        plt.title("subject frequency number: " + str(i))
        plt.xlim([0, len(dict)])
        plt.show()
        i += 1
    labels = np.array(['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])

In [20]:
def analyze_file(lda_model, file):
    dct = Dictionary([file])  # fit dictionary
    bow = dct.doc2bow(file)
    doc_lda = lda_model[bow]
    doc_lda.sort(key=lambda x: x[1], reverse=True)
    print(doc_lda)
    return doc_lda

In [21]:
def get_files_dist(path, file_list, f_model, n_topics):
    all_docs = []
    file_index = 0
    # 5 best topics
    all_dicts = [dict() for i in range(5)]
    for i in range(5):
        for j in range(n_topics):
            all_dicts[i][str(j)] = 0
    file_names = [name for name in listdir(path) if isfile(join(path, name))]
    for name, f in zip(file_names, file_list):
        print("file number: " + str(file_index) + ", file name: " + name)
        doc = analyze_file(f_model, f)
        all_docs.append(doc)
        for i in range(5):
            all_dicts[i][str(doc[0][0])] += 1
        # print(BORDER)
        file_index += 1
    print()
    return all_dicts, all_docs

In [22]:
def save_processed_files(file_list, doc_name, processed_folders_path):
    f = open(processed_folders_path + "\\" + doc_name + ".txt", "w")
    for file in file_list:
        for word in file:
            f.write(word)
            f.write(" ")
        f.write("\n")
    f.close()

In [23]:
def upload_from_file(doc_name, processed_folders_path):
    file_list = []
    f = open(processed_folders_path+ "\\" + doc_name + ".txt")
    for line in f.readlines():
        file = [word for word in line.split(" ")]
        file_list.append(file)
    return file_list

In [24]:
def process_all_files(paths, saving_folder=PROCESSED_FILES_FOLDER):
    for i in range(len(paths)):
        p_files = create_data(paths[i])
        save_processed_files(p_files, names[i], saving_folder)

In [29]:
if __name__ == "__main__":
    case = KUNARAC
    name = ""
    spec = "all"
    dir_path = DATA_PATH + case + "\\all"
    data_name = case + "_" + spec
    n_topics, n_words, n_iterations = 3, 20, 10000
    save = True
    models = ["all", "judgement", "all_and_judgement", "defense", "prosecution", "victims"]
    files = create_data("C:\\Users\\thiya\\OneDrive\\NUS\\Year 3\\Sem 2\\RA\\Martin Buber\\Data")
    model = create_lda_model(files, num_topics=n_topics, iterations=n_iterations)


17.05.16-CAR-OTP-PPPP-0925_MPED2.txt


CalledProcessError: Command 'C:\Users\thiya\OneDrive\NUS\Year 3\Sem 2\RA\Martin Buber\Mallet\bin\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\thiya\AppData\Local\Temp\e6bb17_corpus.txt --output C:\Users\thiya\AppData\Local\Temp\e6bb17_corpus.mallet' returned non-zero exit status 1.

In [129]:
cleaned_data=create_data(DATA_PATH)

17.05.16-CAR-OTP-PPPP-0925_MPED2.txt


In [27]:
print(cleaned_data)

[['questioned', 'by', 'm', 'bala', 'gaye', 'q', 'dr', 'reicherter', 'thank', 'you', 'again', 'for', 'your', 'presence', 'a', 'you', 're', 'welcome', 'q', 'i', 'just', 'wanted', 'to', 'touch', 'upon', 'something', 'that', 'mr', 'raised', 'in', 'relation', 'to', 'post', 'traumatic', 'stress', 'disorder', 'and', 'in', 'particular', 'page', 'to', 'of', 'your', 'report', 'and', 'if', 'i', 'can', 'just', 'read', 'out', 'the', 'extract', 'a', 'is', 'this', 'i', 'have', 'is', 'this', 'just', 'page', 'q', 'could', 'be', 'i', 'm', 'working', 'with', 'the', 'evidence', 'number', 'but', 'just', 'a', 'second', 'let', 'me', 'check', 'that', 'would', 'be', 'page', 'a', 'yeah', 'okay', 'q', 'to', 'and', 'one', 'of', 'the', 'thing', 'that', 'you', 'stated', 'is', 'the', 'change', 'in', 'the', 'brain', 'structure', 'and', 'function', 'that', 'are', 'associated', 'with', 'traumatic', 'stress', 'a', 'yes', 'q', 'exposure', 'can', 'result', 'in', 'cognitive', 'emotional', 'and', 'behavioural', 'difficulty'

In [57]:
create_lda_model(cleaned_data, case="default", num_topics=10, iterations=5000, save=True)

CalledProcessError: Command 'C:\Users\thiya\OneDrive\NUS\Year 3\Sem 2\RA\Martin Buber\Mallet\bin import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\thiya\AppData\Local\Temp\fd8f57_corpus.txt --output C:\Users\thiya\AppData\Local\Temp\fd8f57_corpus.mallet' returned non-zero exit status 1.